## 目录
* [1.数据集介绍](#1-数据集介绍)    
* [2.导入数据](#2-导入数据)
* [3.查看数据信息](#3-查看数据信息)
* [4.使用Keras建立神经网络](#4-使用Keras建立神经网络)
* [5.测试神经网络](#5-测试神经网络)

## 1-数据集介绍
该数据集涵盖了皮马人的医疗记录，以及过去5年内是否有糖尿病，**所有的数据都以数字的形式呈现**。需要解决的问题是，判断一个instance是否有糖尿病（是为1否为0）。这显然是一个**二分类问题**。该数据集中有8个属性及1个类别，表示如下：

* 怀孕次数 --- Number of times pregnant 
* 2小时口服葡萄糖耐量试验中的血浆葡萄糖浓度 --- Plasma glucose concentration a 2 hours in an oral glucose tolerance test 
* 舒张压（毫米汞柱）--- Diastolic blood pressure (mm Hg)
* 2小时血清胰岛素（mu U/ml) --- 2-Hour serum insulin (mu U/ml) 
* 三头肌皮褶厚度 (毫米) --- Triceps skin fold thickness (mm) 
* 体重指数（BMI）--- Body mass index (weight in kg/(height in m)^2)
* 糖尿病血系功能 --- Diabetes pedigree function 
* 年龄（年）--- Age (years)
* 类别：过去5年内是否有糖尿病 --- Class variable (0 or 1)

## 2-导入数据

In [1]:
data_path = '/home/mw/input/damou5913/pima-indians-diabetes.csv'

In [2]:
mv /home/nmic/input/dada7947/resNet50.pb /home/nmic/project

In [3]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np

data = pd.read_csv(data_path, header=None)

## 3-查看数据信息

In [4]:
data.head()

In [5]:
data.shape

In [6]:
data.info()

In [7]:
data.describe()

## 4-使用Keras建立神经网络

### 4.1定义模型
**Keras的模型由层构成**：我们建立一个**Sequential**模型，一层层加入神经元。第一步是确定输入层的数目正确：在创建模型时用```input_dim```参数确定。例如，有8个输入变量，就设成8。

隐层怎么设置？这个问题很难回答，需要慢慢试验。一般来说，如果网络够大，即使存在问题也不会有影响。这个例子里我们用3层全连接网络。

全连接层用Dense类定义：第一个参数是本层神经元个数，然后是初始化方式和激活函数。这里的初始化方法是0到0.05的连续型均匀分布（uniform），**Keras的默认方法也是这个**。也可以用高斯分布进行初始化（normal）。

前两层的激活函数是线性整流函数```relu```，最后一层的激活函数是S型函数```sigmoid```。之前大家喜欢用S型和正切函数，但现在线性整流函数效果更好。为了保证输出是0到1的概率数字，最后一层的激活函数是S型函数，这样映射到0.5的阈值函数也容易。前两个隐层分别有12和8个神经元，最后一层是1个神经元（是否有糖尿病）。

**在这里之所以更新到最新版本的keras，是因为我们按照dataframe的方式读取了数据，old version keras在建模型的时会报错，新版本的keras解决了这个issue。在install完成之后，刷新一下kernel即可**

In [8]:
# !pip install keras==2.1.2

In [9]:
!pip show keras

In [10]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

使用随机梯度下降时最好固定随机数种子，这样你的代码每次运行的结果都一致。这种做法在演示结果、比较算法或debug时特别有效。你可以随便选种子：

In [11]:
seed = 7
np.random.seed(seed)

我们把数据分成输入和输出两组以便交叉检验

In [12]:
X = data.iloc[:,0:8]
Y = data.iloc[:,8]

开始创建模型

In [13]:
model = Sequential()
model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation='relu')) 
model.add(Dense(8, kernel_initializer='uniform', activation='relu'))
model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

我们根据以上代码创建的神经网络结构如下图所示：
<img src="https://cdn.kesci.com/images/lab_upload/1515943659504_47352.png" alt="神经网络模型" style="width: 200px;"/>

### 4.2 编译模型
定义好的模型可以编译：Keras会调用Theano或者TensorFlow编译模型。后端会自动选择表示网络的最佳方法，配合你的硬件，在这里我们Using TensorFlow backend。这步需要定义几个新的参数。训练神经网络的意义是：找到最好的一组权重，解决问题。

我们需要定义损失函数和优化算法，以及需要收集的数据。我们使用```binary_crossentropy```，错误的对数作为损失函数；adam作为优化算法，因为这东西好用。想深入了解请查阅：Adam: A Method for Stochastic Optimization论文。因为这个问题是分类问题，我们收集每轮的准确率。

### 4.3 训练模型
调用模型的```fit()```方法即可开始训练。
网络按轮训练，通过```nb_epoch```参数控制。每次送入的数据（批尺寸）可以用```batch_size```参数控制。**这里我们只跑150轮，每次10个数据**。

In [14]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, Y, epochs=150, batch_size=10)

### 4.4 测试模型
我们把测试数据拿出来检验一下模型的效果。注意这样不能测试在新数据的预测能力。应该将数据分成训练和测试集。

调用模型的```evaluation()```方法，传入训练时的数据。输出是平均值，包括平均误差和其他的数据，例如准确度。

In [15]:
# evaluate the model
scores = model.evaluate(X, Y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

## 5-测试神经网络
深度学习有很多参数要调：大部分都是拍脑袋的。所以测试特别重要：本章我们讨论几种测试方法。本章将：

* 使用Keras进行自动验证
* 使用Keras进行手工验证
* 使用Keras进行K折交叉验证

### 5.1 口算神经网络
创建神经网络时有很多参数：很多时候可以从别人的网络上抄，但是最终还是需要一点点做实验。无论是网络的拓扑结构（层数、大小、每层类型）还是小参数（损失函数、激活函数、优化算法、训练次数）等。

一般深度学习的数据集都很大，数据有几十万乃至几亿个。所以测试方法至关重要。

### 5.2 分割数据
数据量大和网络复杂会造成训练时间很长，所以需要将数据分成训练、测试或验证数据集。Keras提供两种办法：

* 自动验证
* 手工验证

#### 5.2.1 自动验证
Keras可以将数据自动分出一部分，每次训练后进行验证。在训练时用validation_split参数可以指定验证数据的比例，一般是总数据的20%或者33%。

In [16]:
# 我们将上文的代码整理一下，重新复盘并添加自动验证的部分

from keras.models import Sequential
from keras.layers import Dense
import numpy as np

seed = 7
np.random.seed(seed)

data = pd.read_csv(data_path, header=None)

X = data.iloc[:,0:8]
Y = data.iloc[:,8]

model = Sequential()
model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation='relu'))
model.add(Dense(8, kernel_initializer='uniform', activation='relu'))
model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) # Fit the model
model.fit(X, Y, validation_split=0.33, epochs=150, batch_size=10)

scores = model.evaluate(X, Y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

```python
acc: 77.60%
```

#### 5.2.2 手工验证
Keras也可以手工进行验证。我们定义一个```train_test_split```函数，将数据分成2：1的测试和验证数据集。在调用```fit()```方法时需要加入```validation_data```参数作为验证数据，数组的项目分别是输入和输出数据。

In [17]:
# 重新复盘代码，并添加手工验证部分
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

seed = 7
np.random.seed(seed)

data = pd.read_csv(data_path, header=None)

X = data.iloc[:,0:8]
Y = data.iloc[:,8]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=seed)

model = Sequential()
model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation='relu'))
model.add(Dense(8, kernel_initializer='uniform', activation='relu'))
model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=150, batch_size=10)

scores = model.evaluate(X, Y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

```python
acc: 77.21%
```

In [18]:
def da(x,y):
    if x > y:
        return x
    elif x < y:
        return y
    else:
        return x

def dd(a,b):
    if a > b:
        c = da(2,3)+1
        return c
    elif a < b:
        c =  da(3,5)+1
        return c
    else:
        c= da(3,6)+3
        return c

def db():
    k=dd(4,6)
    print (k)

db()

In [19]:
def damou(x):
    k = x * 5678
    print(k)

In [20]:
x=2

In [21]:
damou(x)

In [22]:
!ls /home/mw/work/